In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('/app')

In [3]:
from vexa import VexaAuth, VexaAPI
from chat import UnifiedChatManager, UnifiedContextProvider
from qdrant_search import QdrantSearchEngine
from bm25_search import ElasticsearchBM25
from helpers import jupyter_stream_output
from uuid import UUID
from typing import Optional, List

In [4]:
auth = VexaAuth()
vexa = VexaAPI()
user_id = (await vexa.get_user_info())['id']

# Initialize search engines

Vexa token: 3ae04e20124d40babc5107e658c666b6
User information retrieved successfully.


In [5]:
import os
voyage_apikey = os.getenv('VOYAGE_API_KEY')

In [6]:
from chat import UnifiedContextProvider
from chat import UnifiedChatManager

In [7]:
from qdrant_search import QdrantSearchEngine
from bm25_search import ElasticsearchBM25


In [8]:
qdrant_engine = QdrantSearchEngine(voyage_apikey)
es_engine = ElasticsearchBM25()

Connecting to Elasticsearch at elasticsearch:9200


In [9]:
context_provider = UnifiedContextProvider(qdrant_engine=qdrant_engine,es_engine=es_engine)

In [10]:

@jupyter_stream_output(markdown=True)
async def unified_chat(
    query: str,
    meeting_id: Optional[UUID] = None,
    entities: Optional[List[str]] = None,
    thread_id: Optional[str] = None,
    temperature: float = 0.7
):
    """
    Unified chat function supporting both single-meeting and multi-meeting search
    """
    chat_manager = UnifiedChatManager(
        qdrant_engine=qdrant_engine,
        es_engine=es_engine
    )
    
    async for result in chat_manager.chat(
        user_id=user_id,
        query=query,
        meeting_id=meeting_id,
        entities=entities,
        thread_id=thread_id,
        temperature=temperature
    ):
        if 'chunk' in result:
            yield result['chunk']
        elif 'linked_output' in result:
            yield result

In [12]:
r = await unified_chat(
    query="who are this guys?",
    entities=["AI Marketing Directory"]
)

It seems like you're asking about the individuals mentioned in the context of your meetings. Here are a few key people you have interacted with:

1. **Artem Puzik** - A colleague with whom you frequently discuss technical aspects of your projects, particularly related to the chat application and debugging processes. Your conversations often involve troubleshooting and improving functionality within the system.

2. **Lara Vargas** - You have had discussions with her about influencer marketing strategies and the VEXA product. She has a Brazilian background, which aligns with your interest in obtaining Brazilian permanent residence.

3. **Oleg Ageychev** - Another individual you have spoken with regarding your project, VEXA, and its progress. He has expertise in content marketing and video production.

4. **Sergio Goriachev** - You discussed business ideas and strategies with him, particularly focusing on market opportunities and traffic generation.

5. **Tatiana Sukhova** - You had a conversation with her about social experiences and emotional states, indicating a more personal connection.

6. **Anna Pavlova** - You discussed a mutual acquaintance, Alex Pavlov, who is working on launching multiple startups.

If you need more specific information about any of these individuals or others, please let me know!

In [20]:
r['linked_output']

'In the recent meeting between you, Dmitriy Grankin, and Jasper Vanuytrecht, several key topics were discussed regarding your product related to meeting summaries and knowledge management. Here’s a summary of the main points:\n\n1. **Product Features and User Base**: You highlighted the functionality of your product, which includes a dashboard that provides summaries and a chat interface similar to ChatGPT. This allows users to generate insights based on specific meetings or a collection of meetings. You mentioned that the product is currently in a free trial phase, attracting a stable user base with minimal marketing efforts ([1](/meeting/2a40c363-c52b-4f89-8719-e0be87591485)).\n\n2. **Sales and Marketing Strategy**: Jasper emphasized the importance of identifying target customers and crafting effective outreach messages. He suggested that a strategic approach to sales and marketing is crucial for growth, particularly for tools addressing enterprise privacy and knowledge management ch

In [13]:
meeting_ids = ['q','w']

In [14]:
url_dict = {
    idx+1: f"/meeting/{mid}"
    for idx, mid in enumerate(meeting_ids) if mid
} if meeting_ids else {}

In [15]:
url_dict

{1: '/meeting/q', 2: '/meeting/w'}

In [16]:
import re

In [17]:
async def embed_links(text: str, url_dict: dict) -> str:
    # Find all numeric references in the text
    references = re.findall(r'\[(\d+)\]', text)
    
    # Replace each reference with the corresponding URL based on position
    for idx, ref in enumerate(references, 1):
        if idx in url_dict:
            text = text.replace(f'[{ref}]', f'[{ref}]({url_dict[idx]})')
    return text

In [18]:
await embed_links(r['linked_output'],url_dict)

'In the recent meeting between you, Dmitriy Grankin, and Jasper Vanuytrecht, several key topics were discussed regarding your product related to meeting summaries and knowledge management. Here’s a summary of the main points:\n\n1. **Product Features and User Base**: You highlighted the functionality of your product, which includes a dashboard that provides summaries and a chat interface similar to ChatGPT. This allows users to generate insights based on specific meetings or a collection of meetings. You mentioned that the product is currently in a free trial phase, attracting a stable user base with minimal marketing efforts ([1](/meeting/q)(/meeting/2a40c363-c52b-4f89-8719-e0be87591485)).\n\n2. **Sales and Marketing Strategy**: Jasper emphasized the importance of identifying target customers and crafting effective outreach messages. He suggested that a strategic approach to sales and marketing is crucial for growth, particularly for tools addressing enterprise privacy and knowledge m

In [19]:
chat_manager = SearchChatManager()

NameError: name 'SearchChatManager' is not defined

In [65]:
from helpers import jupyter_stream_output
from typing import Optional

In [ ]:
from typing import Optional
from helpers import jupyter_stream_output
from search import SearchAssistant

# Initialize managers
search_assistant = SearchAssistant()
chat_manager = SearchChatManager()

In [ ]:

@jupyter_stream_output(markdown=True)
async def stream_chat(user_id: str, query: str, thread_id: Optional[str] = None):
    async for result in chat_manager.chat(
        user_id=user_id,
        query=query,
        search_assistant=search_assistant,
        thread_id=thread_id
    ):
        if 'chunk' in result:
            yield result['chunk']
        elif 'output' in result:
            yield result['output']

# Usage
await stream_chat(user_id=user_id, query="андрей")

In [ ]:

# Call the decorated function
await stream_chat(user_id=user_id, query="маркетинг")

In [ ]:
await stream_chat(user_id=user_id,query="маркетинг")

In [ ]:
SearchChatManager

In [21]:
r = await ParsedSearchRequest.parse_request("маркетинг")
queries = r.search_queries

In [ ]:
queries

In [40]:
search_results = [
            await search_assistant.search(q.query, user_id=user_id) 
            for q in queries
        ]

In [41]:
import pandas as pd

In [42]:
search_results = pd.concat(search_results)
search_results = search_results.drop(columns=['vector_scores', 'exact_matches'])\
    .drop_duplicates(subset=['topic_name', 'speaker_name', 'summary', 'details', 'meeting_id'])

# Prepare context using search assistant's methods
prepared_df, indexed_meetings = search_assistant.prep_context(search_results)

In [43]:
columns_to_drop = ['timestamp', 'vector_scores', 'exact_matches', 'source', 'score', 'meeting_id']
existing_columns = [col for col in columns_to_drop if col in prepared_df.columns]
context_prepared = prepared_df.drop(columns=existing_columns)

context = context_prepared.to_markdown(index=False) if not prepared_df.empty else "No relevant context found."
        

In [45]:
context = await context_provider._get_raw_context(user_id=user_id,query="маркетинг")

In [ ]:
context

In [ ]:
context_provider.url_dict

In [ ]:
await search_assistant.search(query="vexa",user_id=user_id)

In [ ]:
context = await context_provider.get_context(user_id=user_id,query="vexa")

In [ ]:
context

In [ ]:

meetings = await vexa.get_meetings()

In [6]:
from chat import MeetingContextProvider,MeetingChatManager

In [7]:
from psql_helpers import get_session,get_meeting_by_id

In [8]:
from helpers import jupyter_stream_output

In [21]:
meeting_id = 'dbf8ad27-649c-4a16-b5a1-5ab3db002bd9'

In [ ]:
from uuid import UUID
meeting_id = UUID(meeting_id)

In [ ]:
meeting_id

In [23]:
query = 'summary'

In [24]:
thread_id = None

In [25]:
from prompts import Prompts
prompts = Prompts()
prompt = prompts.single_meeting_2711

In [29]:
context_provider = MeetingContextProvider([meeting_id])

In [ ]:
async with get_session() as session:
    context = await context_provider.get_context(session=session)
context


In [ ]:
@jupyter_stream_output(markdown=True)
async def chat_with_meeting(thread_id, user_id, meeting_id, query):
    async with get_session() as session:
        context_provider = MeetingContextProvider([meeting_id])
        meeting_chat_manager = MeetingChatManager(session, context_provider=context_provider)
        async for response in meeting_chat_manager.chat(
            meeting_ids=[meeting_id],
            user_id=user_id,
            query=query,
            thread_id=thread_id,
            temperature=0.7,
            prompt=prompts.single_meeting_2711
        ):
            yield response
# Usage
response = await chat_with_meeting(
    thread_id=thread_id,
    user_id='ef7c085b-fdb5-4c94-b7b6-a61a3d04c210',
    meeting_id=meeting_id,
    query="summare meeting"
)

In [ ]:
meeting_

In [ ]:
await meeting_chat_manager.chat(user_id=user_id,query="What was the main topic of the meeting?",meeting_ids=[meeting_id])